# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
vacationpy_df = pd.read_csv("../WeatherPy/Output/raw_data.csv", encoding= "utf-8")
vacationpy_df.dropna(inplace = True) 
vacationpy_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mataura,-46.19,168.86,61.0,71,71,5.01,NZ,1603937847
1,Bluff,-46.60,168.33,61.0,71,72,5.01,NZ,1603937809
2,Kapaa,22.08,-159.32,77.0,83,90,3.36,US,1603937795
3,Busselton,-33.65,115.33,73.0,33,0,10.60,AU,1603937850
4,Provideniya,64.38,-173.30,26.6,68,0,4.47,RU,1603938174


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
gmaps.configure(api_key=g_key)

In [23]:
locations = vacationpy_df[['Lat','Lng']]
humidity = vacationpy_df['Humidity'].astype(float)

In [24]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 100, 
                                 point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
narrow_df = vacationpy_df.loc[
    (vacationpy_df["Max Temp"] > 70) & 
    (vacationpy_df["Max Temp"] < 80) &
    (vacationpy_df["Cloudiness"] == 0) &
    (vacationpy_df["Wind Speed"] < 10), :]
narrow_df = narrow_df.dropna(how='any')
narrow_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,Salalah,17.02,54.09,78.80,74,0,3.36,OM,1603938186
144,Tura,25.52,90.22,75.85,82,0,2.57,IN,1603937678
156,Balotra,25.83,72.23,73.65,27,0,1.19,IN,1603938209
192,Māngrol,21.12,70.12,75.87,67,0,8.97,IN,1603938216
316,Baberu,25.55,80.72,73.54,35,0,1.74,IN,1603938246
321,Kidal,18.44,1.41,75.47,18,0,7.07,ML,1603938247
330,Tenkodogo,11.78,-0.37,75.31,19,0,3.98,BF,1603938249
348,Bentiu,9.23,29.83,74.37,75,0,3.58,SS,1603938255
393,Ayorou,14.73,0.92,79.12,17,0,4.72,NE,1603938266
408,Kamogawa,35.10,140.10,71.01,68,0,8.05,JP,1603938270


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
Hotels = []

narrow_df['Hotel Name'] = ''
narrow_df.head()

params = {
    'types': 'hotel',
    'radius': 5000,
    'key': g_key,
    }

for index, row in narrow_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    
    response = response.json()
    print(response)
    
    try:
        Hotels.append(response['results'][0]['name'])
    except:
        Hotels.append('')
        
narrow_df['Hotel Name'] = Hotels

narrow_df.head()
    

{'html_attributions': [], 'next_page_token': 'AcnaJv0qB4ZugofrZVRkiqeu4omJWsCitomCebrXPlmaKyc6FKuWjcY8wmiFCeN44iqRdg36ZByf_6CBZzL8eU0daGT4rd2LJ7wE9LI5M2J-_fQFowmz1At1tqnxZRbpkQRaU9WI7hxDr6g1TiU7yXluKfYaX5H5Qj6iyU6kun2JBlK9iUxDIXjEqoWv9k_cKi3ugFFXjrfpPWoJ_tZSX6hhtNpmmOMdySJhCrM1lkdpSf9uA8rQ7FOzop1r4nFcMl2_OnPXveoDA79B0ck0CLph-UM8ybAVs6no_kx1_P23SfYeBuFqWaqI-JQgKFe8wta_vr2wQ3lZL-vGpDm6ZpY9oPATj8CtNsiOEEzOyf5njMCOiEvifVadO34Uvg7ZIh5sKe4Hk0Tz_x_jSBWGeNFCI4nSJd71CbP1DwyHMyKYWEQMDzKl', 'results': [{'geometry': {'location': {'lat': 17.0193843, 'lng': 54.11075049999999}, 'viewport': {'northeast': {'lat': 17.1313777, 'lng': 54.240738}, 'southwest': {'lat': 16.9725768, 'lng': 53.9841126}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'name': 'Salalah', 'photos': [{'height': 800, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115346058840123843411">Rayan Saif</a>'], 'photo_reference': 'CmRaAAAAe6ZwiZ7B5fermaOPQhVwNrHKQph9TiXZOPBtwJ0e

{'html_attributions': [], 'next_page_token': 'AcnaJv2LyRR_oJuDPe76cUTnz8DoMhTGv61iXlSRGJTnNzHeDNaTg7RTIPQ-4bBqxfSXaYL7FQ_qMR1AsjmV41GJA0jd1d6Q6_GPApitaAh693duRGCu1MsyWkywibtfbg6FamcWYkSSgYsbmZ-DGnqP6XVFsMhrcs8w759UiJnDNvTKd0reV8D1X-tnFz_OFIaw66KK7epW_jNjlQhG40jS6Q86xvQb-xz_lQB-tZiZyutQ2WVdnfjRRx7-WReh9zhgvS4VJpc6QB0yQjJv5a_Kpcw42ndHmG8M4P9UGXJra-x1PsyKu_PasvIsGSKNE-fKk4Q_BRjW-dPsO4GWwRxNh6Ir4ZOcwbpfWdEEfBwY6APp3zTtdED5wz1cI64wZZKK6AOiv7ORyW6a_tfvhVnPj3Qlmc2fg8IVN3i87QzAa-w5P8q_', 'results': [{'geometry': {'location': {'lat': 25.5140712, 'lng': 90.2032141}, 'viewport': {'northeast': {'lat': 25.5621875, 'lng': 90.2362919}, 'southwest': {'lat': 25.4885296, 'lng': 90.15801429999999}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'name': 'Tura', 'photos': [{'height': 480, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116259507051428984247">Bhoopendra Shrivastava</a>'], 'photo_reference': 'CmRaAAAAL4INW6-Q7F2Eh3M2Ie86SJhkdgfP_9

{'html_attributions': [], 'next_page_token': 'AcnaJv0HvO5OIVp3KeZHgqqTbumfy1JHZk6A2rfyjxTs2QgbffW0_z0jSktCt-dfX8SFPx7hEzBmkCJsMp-wyPlsJT49pCU6ubbbC9ImCIlYASP9SdCl7hvfXmsRK4aWRKwdxeQGjkE64WZ2bfUgURRUHd4DA1uzhooCfJDUdub7a4FeN_q1cHlp_a0WOkVTfUrGkiuPyg-jEcwWPOMJ5lxuALYN2zjQkh5Cn0KLN-Zp1y0KLua9hhlbIq-Sb3GHkcwdYyg5G1fK6gCOj6sCVrIkgZxnhF1f6QniVlqTZQgEMUeWybNwf3DHrc7QCl17157YParzwJWb9BTD0rQ09L6Jg3syBjRgIslJJ6Zn6ALiJoX2D9c5lbyagx5la3CGcSpcKWn43_4lz-O0kxEgrS_t4O29A45iSt-pQ7oOu0Dk-k5rBhJT', 'results': [{'geometry': {'location': {'lat': 25.8302463, 'lng': 72.24556989999999}, 'viewport': {'northeast': {'lat': 25.8697465, 'lng': 72.2801041}, 'southwest': {'lat': 25.7907724, 'lng': 72.198801}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'name': 'Balotra', 'photos': [{'height': 1280, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117912737338649193109">Bheraram Nehra</a>'], 'photo_reference': 'CmRaAAAAR8-iPBJWSWs4-n7piXT8JA8cWnnBoWIALFJ

{'html_attributions': [], 'next_page_token': 'AcnaJv1cSSd3-5rTOxohLEpSuAYK96U9x7t01xLRqji_dRkbMUy4b7Xanzv51CiJeuiyqUFw8nR4zhZCk_UO1ixOOxSp__eyIZtnFUUx_U7OsJuoEZC2HvUd5HkZtWgPZIKkTptS6SQZfIRbsiE8poO5M61C8NzdrvWFYsY2SX_3T-gNxLKUTmJEAqRtQjhnGGEExnbL_Ir3RMpvuRyExdSr-7dslurG7erxp8i94Y9dayG63GQTOUj9vI-Ma7vhhu2mds7OEOH7ffdQV5eB9B69nIuCavZzozH7M_Oyl_S6ptLsX0s7-_41Q9AbMRmKmYKMlj38ZKiI4FtbAXIus-GbbjF1w7TH0mn_MabuDbGLJHKpolrtuDOmFe0IVp_mP-WVp4MdjriVrGz7MGhtxea7J6b2ikUVPkiHo4E6_Us9fbsXypoJ', 'results': [{'geometry': {'location': {'lat': 21.1171698, 'lng': 70.11581129999999}, 'viewport': {'northeast': {'lat': 21.141109, 'lng': 70.14069909999999}, 'southwest': {'lat': 21.0946005, 'lng': 70.089748}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'name': 'Mangrol', 'photos': [{'height': 2184, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117921216778333320793">Bhanesh Bhadrecha</a>'], 'photo_reference': 'CmRaAAAAun-1g7eVf5mpsMalsuyETWTcuN

{'html_attributions': [], 'next_page_token': 'AcnaJv2hX048cmALFA3mr11RfVQzrVPw-LpkbRKTgSPczKfVj9Vkx4X9Lc8n-TWf0X1h00KhvUi_1zvOSJetAOIWsaX--c1CB-HTyYcjXealNx54GEOiMhpi3nukmoBkB9rvgq3f1xv3q07_l9RZmtZUDklNzf-ycZ5SMa-PKeTfsZC2j4_eC_XyGJPUbk3JbjHGdvbMki3-dZraOnolAiNU1ecMuOuSf5v7lKcK2vWagfvUTU3yCLhx7ibWCBLg3Vpd72nNJ7VuLZRCoe0RJ_4kBt7hz5BQQ51MV06z85j3Wq5225479_8fNZOjQxMD5ysKp3VRQzYjqe4SVIDu0a1NP16sacDkwayfTQO42_7wSIF2kOczmVVpdfUY1GnqINwqMurbvBUkPYtbvgSyaL-ra4U5Kd-5Xmh4uCO6pfAoLwibHxhL', 'results': [{'geometry': {'location': {'lat': 25.5503485, 'lng': 80.70480839999999}, 'viewport': {'northeast': {'lat': 25.5594776, 'lng': 80.7175828}, 'southwest': {'lat': 25.5342323, 'lng': 80.6850529}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'name': 'Baberu', 'photos': [{'height': 2340, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109873736628107030639">saddam mohammad Artist</a>'], 'photo_reference': 'CmRaAAAApBFV6GJEJE0J9ZdyANSYijOOLHo

...

{'html_attributions': [], 'next_page_token': 'AcnaJv3OCNcwpxAScDdYwx-35TRipQ_AUMfk-IPbjpVhVvZBxUJfMOx56VEf4GN5XbKMrcloBBcL-kZB2JJ9Ir1QNMf1krGwPvwUCrttGsTdu2V2QeFdaQtN9WdEslvLSExdpBwSf97rXGJ1Jh5Nf7bi2-Up70U5_fUb0hsPBRrO21rXqDhK0ddnEq-ZMjiUvXwUsqq7qyHpe1b3jqWHRxXhmyQ6iOukTySn-DtxYbRiY1UX0-nEUaIVVjUwbdtMTeWqSIHfG4CcoyxBIkFEDu95C9eSp7Fjvb3LaMIm_OWTm16HLR-uXVg7_o5hXrwnAjh6SdlEMdy4r-LdF62IjRhKAIb3E1B2XUgFTiCCs32XXlzfS2Ivgw3ktGIOyT2JQeOh93W7zsOmhSAW3OvPYIBj_95IU8jAvdcIyOmFaJErEqJlbSrz', 'results': [{'geometry': {'location': {'lat': 21.3890824, 'lng': 39.8579118}, 'viewport': {'northeast': {'lat': 21.5930031, 'lng': 40.0028579}, 'southwest': {'lat': 21.2790277, 'lng': 39.6902353}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'name': 'Mecca', 'photos': [{'height': 575, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115102333318772858019">PRO Gamer</a>'], 'photo_reference': 'CmRaAAAAcIb9i0EkJYfPMMWWOfh8xiPCOFxgTi5fxpcZGXfXNlodhDRsp

{'html_attributions': [], 'next_page_token': 'AcnaJv16P1Me6LHqbDHr7DBwffxxQhN_MXvHJX38of6nTaTRfG75Grt-HHBzB8-FIVtKSWVAREdaSR4qIUsVvS0ZvO9rSNH8LGt_z9TQXsSI5zlgQCfxWH3vcZoSrnSXQLLflJ0_Bribs1R1WYcPXUNYx9i2suzsnbWPOoD_1AoJ63PinMzrJ5AtKck_PDhtjcYdvUQrZbHkIweR6t-qbMzkLJ6LZJ3luZDWHj7MRTKpcrVIxYx0so5ylZCXpPqQgIZPp0gQNZPeuj5rfGLK0q-COzKwguEL79RjattVWeU553BtTKzP0tPe7IgBFm3e-qOqQgghJbpSYL4kHVewdFBKuEtHo9mGt_VT3ZiFNDgt11NlAFQTvmkeXp1JNg6N8Ia-5HXPOG10bFR3AO-gkFuRaeO43qPhCG9gU0fDxUAYUjfhWARc', 'results': [{'geometry': {'location': {'lat': 23.354091, 'lng': 116.681972}, 'viewport': {'northeast': {'lat': 23.5409276, 'lng': 116.9796753}, 'southwest': {'lat': 23.1336808, 'lng': 116.5141296}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'name': 'Shantou', 'photos': [{'height': 2204, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/110300409778271939033">wing tai Kwan</a>'], 'photo_reference': 'CmRaAAAAYHZrEiXHzSOm5Bm3Vj9ymSutsYJoZdVyZg1dwMith

{'html_attributions': [], 'next_page_token': 'AcnaJv0Cv95-M4rCPvwe_bZ07d4Br_b804884xIZxcIkX7ZJ2c_rPQ8_idPuUGY2Kd96AkJmgWfbDwR6Oh1Zzwjg2408Upm2Waa_Hyk_r6yK0q4BQjN_USP-ZAJIRw9zbDoVEa3omNgEUtQchSdnSY0QH0swAz5Q_YCsq6JmI57rieeab4gtUfz16BTRKu1d1g7XC8XCgB8bct968Egaf6E-r9Qdtq9_GWOWc7-FEcwYXie0YOb6ZY0gShO-ixVxfZQBIOUHVRmmUe46UVuj-8OZyPgVgy3_76VffgLHrzwH477umd5PAMqS5Uxqs4Sm_hUj8nyDsvGyrgQTKJen7wJ5dIgh6MS-BiHoE6sGdDgNfD-fG7Y6HGxVQcY4IQ4NGLetv28_D-9bn_cKzzV1coNZJaqOprUrVjF59pgp41cG-3pRYfji', 'results': [{'geometry': {'location': {'lat': 24.1299447, 'lng': 67.453141}, 'viewport': {'northeast': {'lat': 24.1476933, 'lng': 67.47000229999999}, 'southwest': {'lat': 24.1220802, 'lng': 67.4442959}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'name': 'Keti Bandar', 'photos': [{'height': 622, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103261507297958124648">Abdullah Moosa</a>'], 'photo_reference': 'CmRaAAAADeuZvyZ1z4lATBdl4qVc35F5O4ufw1Gx

{'html_attributions': [], 'next_page_token': 'AcnaJv2-xIee8Y27EizmRzeNzqaVmEJEMRCi02yjgjzhk663dU4rTp3bxpW6mmQy8HdHHiBicyRoE5ihVGVguuQZTvspi4NLU7xGWbJ_aL1EO_z72YWo0m1SWhGcipnZ2KiYlccbMBCoP-tk5qyWKnmNBWi3r_wtCCPATRHYyNAIOBYe6gp-fm736MqJQCuymQ8werkKNjKcJhe0Tn7keAE9cipX5dPsNy8r_XFn4fdZ8BXaX29Va0oJXGb9TNlNv-tCHx92iZ8wypTeGBuOmC2PqvwWrxfG0cec58ZYbX47xJ6RT3x1xwhWF1oiTqxRhIyP_V32OtoxP40WrQuHJt2wwHjw-SCxbAkCxJESYEQRYpvw7fGzX4Md1Cj_3yzlslWehBB_Q88Voj56NoTOzojx0nI0kNCs9v5DCUdkpvrTcP_-taQG', 'results': [{'geometry': {'location': {'lat': 12.4317585, 'lng': 4.1956212}, 'viewport': {'northeast': {'lat': 12.4752526, 'lng': 4.217667}, 'southwest': {'lat': 12.4275364, 'lng': 4.1775946}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png', 'name': 'Birnin Kebbi', 'photos': [{'height': 2560, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113189760384613687723">Tasiu Aminu</a>'], 'photo_reference': 'CmRaAAAAx2khIVL74uhC1Ux_wjlSrL1He93C3K6Tu-u7e4NNyGd

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
43,Salalah,17.02,54.09,78.80,74,0,3.36,OM,1603938186,Salalah
144,Tura,25.52,90.22,75.85,82,0,2.57,IN,1603937678,Tura
156,Balotra,25.83,72.23,73.65,27,0,1.19,IN,1603938209,Balotra
192,Māngrol,21.12,70.12,75.87,67,0,8.97,IN,1603938216,Mangrol
316,Baberu,25.55,80.72,73.54,35,0,1.74,IN,1603938246,Baberu


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrow_df.iterrows()]
locations = narrow_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))